<a href="https://colab.research.google.com/github/burakulas/kepbinclass/blob/master/kepbinclass_v2b.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# check the version of TensorFlow
from packaging import version

import tensorflow as tf
from tensorflow import keras
print("TensorFlow version: ", tf.__version__)
assert version.parse(tf.__version__).release[0] >= 2, \
    "This notebook requires TensorFlow 2.0."

TensorFlow version:  2.0.0


In [0]:
# import necessary modules
import numpy as np
from numpy.random import seed
seed(42)#for repoductivity
import random as rn
rn.seed(12345)#
import matplotlib.pyplot as plt
import tensorflow
tensorflow.random.set_seed(1234)#rp
import os
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import Activation, Dropout, Flatten, Dense
session_conf = tensorflow.compat.v1.ConfigProto(intra_op_parallelism_threads=1,
                              inter_op_parallelism_threads=1) #rp
from tensorflow.keras import backend as K
from tensorflow.keras import regularizers
from tensorflow.keras import optimizers
from google.colab import files

tensorflow.compat.v1.set_random_seed(1234) #rp

sess = tensorflow.compat.v1.Session(graph=tensorflow.compat.v1.get_default_graph(), config=session_conf) #rp
tensorflow.compat.v1.keras.backend.set_session(sess) #rp

img_width, img_height = 640, 480 # size of images

# train and validation directory
train_data_dir = 'drive/My Drive/newbin/newdata2/train'
validation_data_dir = 'drive/My Drive/newbin/newdata2/validation'
n_train = 1458 # number of train images
n_valid = 294 # number of validation images
epoch = 50 # number of epochs
batch = 16 # batch size

if K.image_data_format() == 'channels_first':
    input_shape = (3, img_width, img_height)
else:
    input_shape = (img_width, img_height, 3)

# the model
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3))) 
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(32))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(3))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

# augmentation configuration
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch,
    class_mode='categorical')

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch,
    class_mode='categorical')

history = model.fit(
    train_generator,
    steps_per_epoch=n_train // batch,
    epochs=epoch,
    validation_data=validation_generator,
    validation_steps=n_valid // batch, shuffle=False)

#saving model

model.save('drive/My Drive/newbin/model/mod_kepbinclass.h5')

# Prediction - a specific file

img_width, img_height = 640, 480
img = image.load_img('drive/My Drive/newbin/newdata2/pred/co1.png', 
                     target_size = (img_width, img_height))
img = image.img_to_array(img)
img = np.expand_dims(img, axis = 0)

pred = model.predict_classes(img) # class prediction
#pred = model.predict_proba(img) # probability prediction
print("---Prediction-----")
print(pred)

# Prediction - images in a folder

pred_data_dir = 'drive/My Drive/newbin/newdata2/pred'

images = []
for img in os.listdir(pred_data_dir):
    imgn = img
    img = os.path.join(pred_data_dir, img)
    img = image.load_img(img, target_size=(img_width, img_height))
    img = image.img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = img / 255
    classp = model.predict_classes(img)
    print(imgn,classp)
